In [ ]:
!nvidia-smi

Mon May  5 17:57:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade datasets transformers accelerate trl[sft]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# 2.1 Load
tldr_ds = load_dataset("trl-lib/tldr")

# 2.2 Inspect splits and column names
print(tldr_ds)
# DatasetDict({
#     train: Dataset({ features: ['prompt','completion'], num_rows: … }),
#     validation: Dataset({ … }),
#     test: Dataset({ … })
# })

print(tldr_ds["train"].column_names)
# ['prompt', 'completion']

# 2.3 Peek at one example
print(tldr_ds["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/116722 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6447 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6553 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 116722
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 6447
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 6553
    })
})
['prompt', 'completion']
{'prompt': "SUBREDDIT: r/relationships\n\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\n\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going 

In [ ]:
from datasets import load_dataset

tldr_ds   = load_dataset("trl-lib/tldr")
train_ds  = tldr_ds["train"]       # 116,722 examples
valid_ds  = tldr_ds["validation"]  #   6,447 examples
test_ds   = tldr_ds["test"]        #   6,553 examples

print(train_ds.column_names)  # ['prompt', 'completion']
print(train_ds.num_rows, valid_ds.num_rows, test_ds.num_rows)


['prompt', 'completion']
116722 6447 6553


In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-base"
tokenizer  = AutoTokenizer.from_pretrained(model_name)

def tokenize_batch(batch):
    # inputs
    tok = tokenizer(
        batch["prompt"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    # targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["completion"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    tok["labels"] = labels["input_ids"]
    return tok

# Use batched=True so this streams and doesn’t error
train_tok = train_ds.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=train_ds.column_names
)
valid_tok = valid_ds.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=valid_ds.column_names
)
test_tok = test_ds.map(
    tokenize_batch,
    batched=True,
    batch_size=1000,
    remove_columns=test_ds.column_names
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/116722 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/6447 [00:00<?, ? examples/s]

Map:   0%|          | 0/6553 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset

# Load the prefs split you want
prefs = load_dataset("UCL-DARK/openai-tldr-summarisation-preferences")["train"]

# Inspect to confirm…
print(prefs.column_names)  # ['info','split','summaries','choice','worker','batch']
print(prefs[0]['info'])    # to see the fields under info

# Remap using info['title'] & info['post'], and pick the summary at index choice
def preprocess_prefs(example):
    info    = example["info"]
    title   = info["title"]
    post    = info["post"]
    prompt  = f"TITLE: {title}\nPOST: {post}\nTL;DR:"
    # use the worker’s preferred summary
    summary = example["summaries"][example["choice"]]["text"].strip()
    return {"prompt": prompt, "completion": summary}

# Apply
test_prefs = prefs.map(
    preprocess_prefs,
    batched=False,
    remove_columns=prefs.column_names
)

# Verify
print(test_prefs.column_names, test_prefs.num_rows)
print(test_prefs[0])


README.md:   0%|          | 0.00/817 [00:00<?, ?B/s]

train-00000-of-00001.json:   0%|          | 0.00/184M [00:00<?, ?B/s]

validation-00000-of-00001.json:   0%|          | 0.00/67.5M [00:00<?, ?B/s]

test-00000-of-00001.json:   0%|          | 0.00/103M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/92858 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/33083 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50719 [00:00<?, ? examples/s]

['info', 'split', 'summaries', 'choice', 'worker', 'batch']
{'id': 't3_f2bvu', 'post': 'A little background first. Like most engineers I\'m taking differential equations and also solid mechanics. For solid mechanics we have to do a lab in addition to taking the class. Pretty standard stuff.\n\nAnyway, the class had us sign up for labs in groups by providing sign up sheets where we could sign up for which lab we wanted by providing our name and student ID numbers. The lab assignments were posted online today with a column for our names and student ID numbers (which had been blacked out). Somehow, my name wasn\'t on the list and in frustration I began dragging the scroll bar up and down frantically hoping that I\'d magically see my name; that\'s when I saw it.\n\nWhen dragging the scroll bar fast enough, I found that the blacked out student ID numbers would temporarily become visible. At first I thought, "No big deal, what could someone do with a person\'s ID number." Then it hit me. My 

Map:   0%|          | 0/92858 [00:00<?, ? examples/s]

['prompt', 'completion'] 92858
{'prompt': 'TITLE: Averted a shit storm in my math class today and nobody will know (except you)\nPOST: A little background first. Like most engineers I\'m taking differential equations and also solid mechanics. For solid mechanics we have to do a lab in addition to taking the class. Pretty standard stuff.\n\nAnyway, the class had us sign up for labs in groups by providing sign up sheets where we could sign up for which lab we wanted by providing our name and student ID numbers. The lab assignments were posted online today with a column for our names and student ID numbers (which had been blacked out). Somehow, my name wasn\'t on the list and in frustration I began dragging the scroll bar up and down frantically hoping that I\'d magically see my name; that\'s when I saw it.\n\nWhen dragging the scroll bar fast enough, I found that the blacked out student ID numbers would temporarily become visible. At first I thought, "No big deal, what could someone do w

In [ ]:
# Question 6: Tokenize UCL-DARK held-out test set
test_prefs_tok = test_prefs.map(
    tokenize_batch,
    batched=True,
    batch_size=500,
    remove_columns=test_prefs.column_names
)

Map:   0%|          | 0/92858 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tuned_tldr",
    per_device_train_batch_size=8,    # ↑ from 4
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,    # ↓ from 4
    num_train_epochs=3,
    learning_rate=3e-5,
    bf16=True,                        # switched from fp16
    logging_steps=100,
    save_total_limit=2,
    dataloader_num_workers=4,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 4.1 Fine-tune
trainer.train()

# 4.2 Manual evaluation
print("Validation:",   trainer.evaluate(valid_tok))
print("Held-out test:", trainer.evaluate(test_prefs_tok))

<ipython-input-12-b8c4971f73ed>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vihaan-shah2014 (vihaan-shah2014-rutgers-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,9.626400
200,1.084200
300,0.758700
400,0.746100
500,0.729300
600,0.732200
700,0.719300
800,0.716000
900,0.733100
1000,0.723400


Validation: {'eval_loss': 0.6332863569259644, 'eval_runtime': 41.4487, 'eval_samples_per_second': 155.542, 'eval_steps_per_second': 19.446, 'epoch': 2.9996573230073333}
Held-out test: {'eval_loss': 0.48109379410743713, 'eval_runtime': 588.6449, 'eval_samples_per_second': 157.749, 'eval_steps_per_second': 19.72, 'epoch': 2.9996573230073333}


In [ ]:
trainer.save_model("tuned_tldr/final")
tokenizer.save_pretrained("tuned_tldr/final")

('tuned_tldr/final/tokenizer_config.json',
 'tuned_tldr/final/special_tokens_map.json',
 'tuned_tldr/final/spiece.model',
 'tuned_tldr/final/added_tokens.json',
 'tuned_tldr/final/tokenizer.json')

In [ ]:
# 1. Create a clean folder (optional if it doesn’t already exist)
import os
os.makedirs("saved_model", exist_ok=True)

# 2. Save the model weights + config
trainer.save_model("saved_model")       # writes pytorch_model.bin, config.json, etc.

# 3. Save the tokenizer files
tokenizer.save_pretrained("saved_model")  # writes vocab, merges (if BPE), tokenizer_config.json, etc.

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/spiece.model',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [ ]:
# 1. Re-save to ensure folder is up-to-date
trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_model")

# 2. Create a ZIP archive of the folder
!zip -r saved_model.zip saved_model

# 3. Download the ZIP to your local machine
from google.colab import files
files.download("saved_model.zip")

  adding: saved_model/ (stored 0%)
  adding: saved_model/special_tokens_map.json (deflated 85%)
  adding: saved_model/spiece.model (deflated 48%)
  adding: saved_model/tokenizer.json (deflated 74%)
  adding: saved_model/config.json (deflated 62%)
  adding: saved_model/model.safetensors (deflated 7%)
  adding: saved_model/training_args.bin (deflated 51%)
  adding: saved_model/generation_config.json (deflated 29%)
  adding: saved_model/tokenizer_config.json (deflated 95%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>